In [1]:
using Revise
using ITensors
using ITensorTDVP

In [2]:
import ITensors: leftlim, rightlim, setleftlim!, setrightlim!

function ITensors.replacebond!(M::MPO, b::Int, phi::ITensor; kwargs...)
    ortho::String = get(kwargs, :ortho, "left")
    swapsites::Bool = get(kwargs, :swapsites, false)
    which_decomp::Union{String,Nothing} = get(kwargs, :which_decomp, nothing)
    normalize::Bool = get(kwargs, :normalize, false)
  
    indsMb = inds(M[b])
    if swapsites
      error("Not implemented")
    end 
  
    L, R, spec = factorize(
      phi, indsMb; which_decomp=which_decomp, tags=tags(linkind(M, b)), kwargs...
    )
  
    M[b] = L 
    M[b + 1] = R 
    if ortho == "left"
      leftlim(M) == b - 1 && setleftlim!(M, leftlim(M) + 1)
      rightlim(M) == b + 1 && setrightlim!(M, rightlim(M) + 1)
      normalize && (M[b + 1] ./= norm(M[b + 1]))
    elseif ortho == "right"
      leftlim(M) == b && setleftlim!(M, leftlim(M) - 1)
      rightlim(M) == b + 2 && setrightlim!(M, rightlim(M) - 1)
      normalize && (M[b] ./= norm(M[b]))
    else
      error(
        "In replacebond!, got ortho = $ortho, only currently supports `left` and `right`."
      )   
    end
    return spec
  end

In [3]:
#==
import ITensors: AbstractProjMPO, OneITensor, site_range

function ITensors.contract(P::AbstractProjMPO, v::ITensor)::ITensor
    itensor_map = Union{ITensor,OneITensor}[lproj(P)]
    append!(itensor_map, P.H[site_range(P)])
    push!(itensor_map, rproj(P))

    #for i in eachindex(itensor_map)
      #println("itensor_map ", i, "  ", inds(itensor_map[i]))
    #end
  
    # Reverse the contraction order of the map if
    # the first tensor is a scalar (for example we
    # are at the left edge of the system)
    if dim(first(itensor_map)) == 1
      reverse!(itensor_map)
    end 
  
    # Apply the map 
    Hv = v 
    for it in itensor_map
      Hv *= it
    end 
    return Hv
  end
  ==#

In [4]:
nbit = 5
sites = siteinds("Qubit", nbit)

M1 = randomMPO(sites)
M2 = randomMPO(sites)

# Fix siteinds manually :)
M1 = replaceprime(M1, 1=>2, 0=>1)
#@show M1
#@show M2

M12 = contract(M1, M2; alg="fit")

inds(init_mps[n]) = ((dim=2|id=922|"Qubit,Site,n=1")'', (dim=2|id=922|"Qubit,Site,n=1"), (dim=1|id=800|"Link,l=1"))
inds(init_mps[n]) = ((dim=2|id=956|"Qubit,Site,n=2")'', (dim=2|id=956|"Qubit,Site,n=2"), (dim=1|id=473|"Link,l=2"), (dim=1|id=800|"Link,l=1"))
inds(init_mps[n]) = ((dim=2|id=605|"Qubit,Site,n=3")'', (dim=2|id=605|"Qubit,Site,n=3"), (dim=1|id=409|"Link,l=3"), (dim=1|id=473|"Link,l=2"))
inds(init_mps[n]) = ((dim=2|id=383|"Qubit,Site,n=4")'', (dim=2|id=383|"Qubit,Site,n=4"), (dim=1|id=872|"Link,l=4"), (dim=1|id=409|"Link,l=3"))
inds(init_mps[n]) = ((dim=2|id=428|"Qubit,Site,n=5")'', (dim=2|id=428|"Qubit,Site,n=5"), (dim=1|id=872|"Link,l=4"))


MPS
[1] ((dim=2|id=922|"Qubit,Site,n=1"), (dim=2|id=922|"Qubit,Site,n=1")'', (dim=1|id=802|"Link,l=1"))
[2] ((dim=2|id=956|"Qubit,Site,n=2"), (dim=1|id=161|"Link,l=2"), (dim=2|id=956|"Qubit,Site,n=2")'', (dim=1|id=802|"Link,l=1"))
[3] ((dim=2|id=605|"Qubit,Site,n=3"), (dim=1|id=782|"Link,l=3"), (dim=2|id=605|"Qubit,Site,n=3")'', (dim=1|id=161|"Link,l=2"))
[4] ((dim=2|id=383|"Qubit,Site,n=4"), (dim=1|id=493|"Link,l=4"), (dim=2|id=383|"Qubit,Site,n=4")'', (dim=1|id=782|"Link,l=3"))
[5] ((dim=2|id=428|"Qubit,Site,n=5"), (dim=2|id=428|"Qubit,Site,n=5")'', (dim=1|id=493|"Link,l=4"))


In [5]:
M12_ref = contract(M1, M2; alg="naive")

MPS
[1] ((dim=2|id=922|"Qubit,Site,n=1")'', (dim=2|id=922|"Qubit,Site,n=1"), (dim=1|id=121|"CMB,Link"))
[2] ((dim=2|id=956|"Qubit,Site,n=2")'', (dim=2|id=956|"Qubit,Site,n=2"), (dim=1|id=467|"CMB,Link"), (dim=1|id=121|"CMB,Link"))
[3] ((dim=2|id=605|"Qubit,Site,n=3")'', (dim=2|id=605|"Qubit,Site,n=3"), (dim=1|id=353|"CMB,Link"), (dim=1|id=467|"CMB,Link"))
[4] ((dim=2|id=383|"Qubit,Site,n=4")'', (dim=2|id=383|"Qubit,Site,n=4"), (dim=1|id=758|"CMB,Link"), (dim=1|id=353|"CMB,Link"))
[5] ((dim=2|id=428|"Qubit,Site,n=5")'', (dim=2|id=428|"Qubit,Site,n=5"), (dim=1|id=758|"CMB,Link"))


In [6]:
M12_ref ≈ M12

true